In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
bakery1000 = pd.read_csv('/Users/Dima/Desktop/CSC466/apriori/1000/1000-out1.csv',sep = '\n', header = None)
bakery1000Mtemp = bakery1000.ix[:,0].values
bakery1000M = [list(map(int, x.split(','))) for x in bakery1000Mtemp]
for x in bakery1000M:
    del x[0]

In [6]:
bakery1000M[1:10]

[[1, 19],
 [1, 19],
 [3, 4, 15, 18, 35, 44],
 [2, 4, 7, 9, 23],
 [14, 21, 44],
 [4, 12, 31, 36, 44, 48],
 [15, 27, 28],
 [2, 28],
 [3, 18, 35]]

In [14]:
I =  list(set(x for l in bakery1000M for x in l))

In [15]:
def support(item, baskets):
    top = sum([1 if item in basket else 0 for basket in baskets])
    bottom = len(baskets)
    return top*1.0/bottom

In [24]:
firstIter = set(filter(lambda a: a != -1, [x if support(x, bakery1000M) >= .09 else -1 for x in I]))

In [25]:
firstIter

{4, 5, 7, 9, 14, 22, 27, 28, 31, 45}

In [ ]:
def getPairs(F, k):
    outList = []
    for first in F:
        for second in F:
            if len(first) == len(second) and len(first) == k:
                outList.append([first, second, set(set(first) | set(second))])
    return outList 